# Social Graphs Project

## Pre-processing

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request
import json
import os
import string  
import math
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import community
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from wordcloud import WordCloud, ImageColorGenerator
from fa2 import ForceAtlas2
from PIL import Image, ImageFont

### Extract wiki data

### Data cleaning

In [188]:
df = pd.read_csv('../git_data_third.csv',index_col=0)
df['languages']=df['languages'].str.replace(' ',',').str.replace('\n','').str.replace("'","")
df['out']=df['out'].str.replace(' ',',').str.replace('\n','').str.replace("'","")
df.head()

,user_name,owner,repos,name,type,company,location,bio,created_at,updated_at,following,description,languages,out
0,josephmisiti,josephmisiti,awesome-machine-learning,Joseph Misiti,User,Math & Pencil,"Brooklyn, NY",Mathematician & Co-founder of Math & Pencil,2010-04-17T21:09:13Z,2020-10-29T18:11:39Z,"['mojombo', 'defunkt', 'vanpelt', 'macournoyer...",SDK and example code for building voice-enabl...,"[C,C++,CSS,Go,Java,JavaScript,Jupyter,Notebook...","[JWarmenhoven,agramfort,alextp,amueller,bamos,..."
1,wepe,wepe,MachineLearning,wepon,User,AntFin,China Hangzhou,NaN,2014-10-10T14:28:18Z,2020-11-05T15:54:10Z,"['pluskid', 'benanne', 'colah', 'kennethreitz4...",1st Place Season one & 6th Place Season two1st...,"[C++,HTML,Java,Python,Shell]","[aymericdamien,enjoyhot,guolinke,junlulocky]"
2,ZuzooVn,ZuzooVn,machine-learning-for-software-engineers,Nam Vu,User,NaN,Vietnam,A Vietnamese Software Engineer who is really p...,2012-02-11T16:55:02Z,2020-10-12T02:40:45Z,"['ole', 'escoz', 'pokeb', 'tcdoan', 'dyang', '...",Android executables browserEasy to use CFNetwo...,"[C,DIGITAL,Command,Language,HTML,Java,JavaScri...",[jwasham]
3,rasbt,rasbt,python-machine-learning-book,Sebastian Raschka,User,UW-Madison,"Madison, WI",Machine Learning researcher & open source cont...,2013-10-05T16:06:10Z,2020-11-09T23:00:46Z,"['ctb', 'ogrisel', 'tacaswell', 'GaelVaroquaux...",My Solutions for the Advent of Code 2016A repo...,"[C,C++,CSS,HTML,Jupyter,Notebook,Lua,OpenEdge,...","[amueller,jakevdp,mblondel,ogrisel,rhiever]"
4,Jack-Cherish,Jack-Cherish,Machine-Learning,Jack Cui,User,Northeastern University,China,:octocat:公众号：JackCui-AI,2016-01-25T03:38:06Z,2020-07-31T13:54:15Z,"['kennethreitz42', 'karpathy', 'marvis', 'shic...",:art:冒泡排序；直接插入排序；希尔排序；快速排序；堆排序；归并排序；基数排序C/C++:...,"[C++,HTML,Python]",[]


In [189]:
def stringToList(colname):
    
    df[colname]=[df[colname][i][1:len(df[colname][i])-1].split(',') for i in range(len(df))]

stringToList('out')
stringToList('following')
stringToList('languages')


## Generate network

In [190]:
# store the network of users with DiGraph
G = nx.DiGraph()
G.add_nodes_from(df['user_name'].values)

for row in df.iterrows():
    
    z = [(row[1]['user_name'], other_node) for other_node in row[1]['out']]
    G.add_edges_from(z)

largest_cc = max(nx.weakly_connected_components(G), key=len)
GCC = G.subgraph(largest_cc)


## Basic statistics

In [191]:
print(f"Number of nodes in GCC: {GCC.number_of_nodes()}")
print(f"Number of edges in GCC: {GCC.number_of_edges()}")

Number of nodes in GCC: 738
Number of edges in GCC: 2301


## Visualization